<a href="https://colab.research.google.com/github/HunbeomBak/nlp_study/blob/main/Untitled4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 글로브(Global Vectors for Word Representation, GloVe)
- 운트 기반과 예측 기반을 모두 사용하는 방법
- 존의 카운트 기반의 LSA(Latent Semantic Analysis)와 예측 기반의 Word2Vec의 단점을 보완


## 기존 방법론에 대한 비판
- LSA
    - 각 단어의 빈도수를 카운트 한 행렬이라는 전체적인 통계 정보를 입력으로 받아 차원을 축소(Truncated SVD)하여 잠재된 의미를 끌어내는 방법
    - 카운트 기반으로 코퍼스의 전체적인 통계 정보를 고려
    - 왕:남자 = 여왕:? (정답은 여자)와 같은 단어 의미의 유추 작업(Analogy task)에는 성능이 떨어짐
    

    

- Word2Vec
    - 실제값과 예측값에 대한 오차를 손실 함수를 통해 줄여나가며 학습하는 예측 기반의 방법
    - 예측 기반으로 단어 간 유추 작업에는 LSA보다 뛰어남
    - 임베딩 벡터가 윈도우 크기 내에서만 주변 단어를 고려 => 퍼스의 전체적인 통계 정보를 반영하지 못함
    
## 윈도우 기반 동시 등장 행렬(Window based Co-occurrence Matrix)
- 단어의 동시 등장 행렬
    -  행과 열을 전체 단어 집합의 단어들로 구성하고, i 단어의 윈도우 크기(Window Size) 내에서 k 단어가 등장한 횟수를 i행 k열에 기재한 행렬
    
    
## 동시 등장 확률(Co-occurrence Probability)
- 동시 등장 확률 P(k | i)는 동시 등장 행렬로부터 특정 단어 i의 전체 등장 횟수를 카운트하고, 특정 단어 i가 등장했을 때 어떤 단어 k가 등장한 횟수를 카운트하여 계산한 조건부 확률
- P(k | i) 에서 i를 중심 단어(Center Word), k를 주변 단어(Context Word)라고 했을 때, 동시 등장 행렬에서 중심 단어 i의 행의 모든 값을 더한 값을 분모로 하고 i행 k열의 값을 분자로 한 값

## 손실 함수(Loss function)
- GloVe의 목표 : 임베딩 된 중심 단어와 주변 단어 벡터의 내적이 전체 코퍼스에서의 동시 등장 확률이 되도록 만드는 것

## GloVe 훈련시키기
### 설치

In [1]:
!pip install glove_python


     |████████████████████████████████| 266kB 8.1MB/s 
  Created wheel for glove-python: filename=glove_python-0.1.0-cp36-cp36m-linux_x86_64.whl size=700255 sha256=92b92ef3d916bc4780872c12ad39f6e9c0b9e52d10fff8b47dd1768adca9427b
  Stored in directory: /root/.cache/pip/wheels/88/4b/6d/10c0d2ad32c9d9d68beec9694a6f0b6e83ab1662a90a089a4b
Successfully built glove-python


### 훈련 데이터 습득 및 전처리

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

In [4]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7fb51bb50e10>)

In [5]:
targetXML=open('ted_en-20160408.xml', 'r', encoding='UTF8')
# 저자의 경우 윈도우 바탕화면에서 작업하여서 'C:\Users\USER\Desktop\ted_en-20160408.xml'이 해당 파일의 경로.  
target_text = etree.parse(targetXML)
parse_text = '\n'.join(target_text.xpath('//content/text()'))
# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.

content_text = re.sub(r'\([^)]*\)', '', parse_text)
# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.

sent_text = sent_tokenize(content_text)
# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.

normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.

result = []
result = [word_tokenize(sentence) for sentence in normalized_text]
# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.

### 훈련

In [6]:
from glove import Corpus, Glove


In [7]:
corpus = Corpus() 
corpus.fit(result, window=5)
# 훈련 데이터로부터 GloVe에서 사용할 동시 등장 행렬 생성

In [8]:
glove = Glove(no_components=100, learning_rate=0.05)
glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
# 학습에 이용할 쓰레드의 개수는 4로 설정, 에포크는 20.

Performing 20 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


### 유사한 단어 출력

In [9]:
model_result1=glove.most_similar("man")
print(model_result1)

[('woman', 0.959524093448378), ('girl', 0.8898895700479628), ('guy', 0.8802885487582404), ('young', 0.8490654248003882)]


In [10]:
model_result2=glove.most_similar("boy")
print(model_result2)

[('girl', 0.9447202693246827), ('woman', 0.8564067784601618), ('man', 0.842407340763357), ('kid', 0.8173278318033197)]


In [11]:
model_result3=glove.most_similar("university")
print(model_result3)

[('harvard', 0.8833352700654012), ('mit', 0.8431742722949984), ('cambridge', 0.8360920494806658), ('stanford', 0.8307473885663621)]


In [12]:
model_result4=glove.most_similar("water")
print(model_result4)

[('air', 0.8336621514907467), ('fresh', 0.8259929432206339), ('clean', 0.819764957258365), ('food', 0.8114595229131326)]


In [13]:
model_result5=glove.most_similar("physics")
print(model_result5)

[('chemistry', 0.8871240061550107), ('economics', 0.8735734929487828), ('mathematics', 0.8690844666663236), ('beauty', 0.8651302875997448)]


In [14]:
model_result6=glove.most_similar("muscle")
print(model_result6)

[('tissue', 0.8485704129952255), ('nerve', 0.8248290602812862), ('channel', 0.7774857980322292), ('skeletal', 0.7742452331505026)]


In [15]:
model_result7=glove.most_similar("clean")
print(model_result7)

[('fresh', 0.8264459707039261), ('water', 0.8197649572583651), ('wind', 0.8019940902233511), ('heat', 0.7832213354139386)]


## Save&load model
### Save

In [16]:
import numpy as np

In [20]:
## 나중에 워드 임베딩 시각화용
np.savetxt('glove-vector.tsv', glove.word_vectors, delimiter='\t') 
with open('glove-metadata.tsv', 'w', encoding='utf-8') as f: 
  for key in glove.dictionary.keys(): f.write(f"{key}\n")



In [21]:
## 모델 저장
glove.save('glove_study.model')

### load

In [22]:
glove = Glove.load('glove_study.model')


In [23]:
model_result1=glove.most_similar("man")
print(model_result1)

[('woman', 0.959524093448378), ('girl', 0.8898895700479628), ('guy', 0.8802885487582404), ('young', 0.8490654248003882)]
